In [213]:
from datetime import *
from datetime import datetime
import pandas as pd
import hashlib
import json
import calendar
from functools import reduce
import os
def read_data(select_toppic):
        list_union = []    
        for path in select_toppic:
            datapath = path       
            namedata_type = datapath.split(".")
            if namedata_type[-1] == 'csv':
                file_read = pd.read_csv(datapath, encoding = 'windows-1254')
            else:
                file_read = pd.read_excel(datapath,index_col=None)
            col_name = list(file_read.head(0))            
            check_head = [head_name for head_name in col_name if "DATE" in head_name.upper()]
            for head_namedate in check_head:
                data_date = [datetime.strftime(date, '%d/%m/%Y') if type(date) == datetime else datetime.strftime(datetime.strptime(date, "%d/%m/%Y").date(), '%d/%m/%Y') for date in file_read[head_namedate]]
                file_read[head_namedate] = data_date               
            list_union.append(file_read)
            
        frame = pd.concat(list_union, ignore_index=True, sort=False)
        col_sort = list(frame.head(0))
        union_frame = frame.sort_values(col_sort)
        dup = union_frame.drop_duplicates()
        set_dup = dup.reset_index(drop=True) 
        return set_dup
        
path = ["C:/Users/ASUS/Documents/vs/16th/test20line.xls"]
data = read_data(path)
data

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2019-152156,11/08/2019,11/11/2019,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2019-152156,11/08/2019,11/11/2019,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2019-138688,06/12/2019,16/06/2019,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2018-108966,10/11/2018,18/10/2018,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2018-108966,10/11/2018,18/10/2018,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
5,6,CA-2017-115812,06/09/2017,14/06/2017,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
6,7,CA-2017-115812,06/09/2017,14/06/2017,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800,4,0.00,1.9656
7,8,CA-2017-115812,06/09/2017,14/06/2017,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520,6,0.20,90.7152
8,9,CA-2017-115812,06/09/2017,14/06/2017,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040,3,0.20,5.7825
9,10,CA-2017-115812,06/09/2017,14/06/2017,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000,5,0.00,34.4700


In [214]:
def dim_meas_rc(row_list,column_list,dimension,measure):
        list_dim = []
        list_meas = []
        list_mark = []
        list_filter = []

        for namerow in row_list:
            list_drill_row = namerow.split("+")
            list_namerow = list_drill_row[0].split(".")
            if len(list_namerow) > 1:
                if list_namerow[-1] in dimension:
                    list_dim.append(list_namerow[-1])
                    list_filter.append(list_namerow[0])
                if list_namerow[-1] in measure:
                    list_meas.append(list_namerow[-1])
                    list_mark.append(list_namerow[0])
            else:
                if list_namerow[0] in dimension:
                    list_dim.append(list_namerow[0])
                    list_filter.append("NONE")
                if list_namerow[0] in measure:
                    list_meas.append(list_namerow[0])
                    list_mark.append("NONE")
                
        for namecolumn in column_list:
            list_drill_col = namecolumn.split("+")
            list_namecolumn = list_drill_col[0].split(".")
            if len(list_namecolumn) > 1:
                if list_namecolumn[-1] in dimension:
                    list_dim.append(list_namecolumn[-1])
                    list_filter.append(list_namecolumn[0])
                if list_namecolumn[-1] in measure:
                    list_meas.append(list_namecolumn[-1])
                    list_mark.append(list_namecolumn[0])
            else:
                if list_namecolumn[0] in dimension:
                    list_dim.append(list_namecolumn[0])
                    list_filter.append("NONE")
                if list_namecolumn[0] in measure:
                    list_meas.append(list_namecolumn[0])
                    list_mark.append("NONE")
        return list_dim,list_meas,list_mark,list_filter

dimensions = ["Row ID","Order ID","Order Date","Ship Date","Ship Mode","Customer ID","Customer Name",
        "Segment","Country/Region","City","State","Postal Code","Region","Product ID","Category","Sub-Category","Product Name"]
measure = ["Sales","Quantity","Discount","Profit"]
row = ["Year.Order Date","MAX.Quantity","Category"]
column = ["Sales","Sub-Category"]

list_dim,list_meas,list_mark,list_filter = dim_meas_rc(row,column,dimensions,measure)
list_meas


['Quantity', 'Sales']

In [215]:
def check_date_filter(file_read,list_dim,list_filter):

    list_dimfil = []
    for s_fil in range(len(list_dim)):
        upper_filter = list_filter[s_fil].upper()
        dim_name = list_dim[s_fil]
        dim_fil = list_filter[s_fil]
        
        if "DATE" in dim_name.upper():
            data_date = [datetime.strptime(date, "%d/%m/%Y").date() if type(date) == str else datetime.date(date)  for date in file_read[dim_name]]
            if  upper_filter == 'YEAR' or upper_filter == 'NONE':
                fil_date = list(pd.DatetimeIndex(data_date).year)
                dim_fil = 'Year'
            if upper_filter == 'QUARTER':
                fil_date = list(pd.DatetimeIndex(data_date).quarter) 
            if upper_filter == 'MONTH':
                fil_date = list(pd.DatetimeIndex(data_date).month)
                # fil_date = [calendar.month_name[int(date_time)] for date_time in list_month]    
            if upper_filter == 'DATE':
                fil_date = list(pd.DatetimeIndex(data_date).day)
            file_read[dim_fil+"."+dim_name] = fil_date
            data_infil = [int(date_time) for date_time in fil_date]   
            list_dimfil.append(dim_fil+"."+dim_name)         
        else:
            file_read[dim_name] = file_read[dim_name]
            list_dimfil.append(dim_name)
        
    frame_data = pd.DataFrame.from_dict(file_read)
    frame_merge = frame_data.drop_duplicates().sort_values(frame_data.keys()[0])
        
    return file_read,list_dimfil

check_datadate,list_dimfil = check_date_filter(data,list_dim,list_filter)
check_datadate

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Year.Order Date
0,1,CA-2019-152156,11/08/2019,11/11/2019,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,2019
1,2,CA-2019-152156,11/08/2019,11/11/2019,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,2019
2,3,CA-2019-138688,06/12/2019,16/06/2019,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,2019
3,4,US-2018-108966,10/11/2018,18/10/2018,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,2018
4,5,US-2018-108966,10/11/2018,18/10/2018,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,2018
5,6,CA-2017-115812,06/09/2017,14/06/2017,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694,2017
6,7,CA-2017-115812,06/09/2017,14/06/2017,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800,4,0.00,1.9656,2017
7,8,CA-2017-115812,06/09/2017,14/06/2017,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520,6,0.20,90.7152,2017
8,9,CA-2017-115812,06/09/2017,14/06/2017,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040,3,0.20,5.7825,2017
9,10,CA-2017-115812,06/09/2017,14/06/2017,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000,5,0.00,34.4700,2017


In [216]:
list_dimfil

['Year.Order Date', 'Category', 'Sub-Category']

In [222]:
def check_mark(file_read,list_dimfil,list_meas,list_mark):
    frame_mark_list = []
    
    group_data = file_read.groupby(list_dimfil,as_index=False)
    for s_mark in range(len(list_mark)):#mark data
        upper_mark = list_mark[s_mark].upper() 
        if  upper_mark == 'MEAN':
            data_s_mark = group_data[list_meas[s_mark]].mean()
        if upper_mark == 'MAX':
            data_s_mark = group_data[list_meas[s_mark]].max()
        if upper_mark == 'MIN':
            data_s_mark = group_data[list_meas[s_mark]].min()
        if upper_mark == 'COUNT':
            data_s_mark = group_data[list_meas[s_mark]].count()
        if upper_mark == 'SUM' or upper_mark == 'NONE':
            data_s_mark = group_data[list_meas[s_mark]].sum()
            list_mark[s_mark] = 'SUM'
        if upper_mark == 'MEDIAN':
            data_s_mark = group_data[list_meas[s_mark]].median()
        
        data_s_mark[list_mark[s_mark]+"."+list_meas[s_mark]] = data_s_mark[list_meas[s_mark]]
        del data_s_mark[list_meas[s_mark]]

        frame_mark_list.append(data_s_mark)

    if len(frame_mark_list) >= 1:
        frame_merge = reduce(lambda left, right: pd.merge(left,right), frame_mark_list)
    else:
        frame_merge = data_s_mark
    
    for n, key in enumerate(frame_merge.keys()): # add item
        list_month = frame_merge[key]
        if "DATE" in key.upper() and "MONTH" in key.upper():
            fil_date = [calendar.month_name[int(date_time)] for date_time in list_month]  
            frame_merge[key]  = fil_date
        elif "DATE" in key.upper() and 'MONTH' not in key.upper():
            fil_data = [str(data) for data in list_month]  
            frame_merge[key]  = fil_data


    return frame_merge
check_markfil = check_mark(check_datadate,list_dimfil,list_meas,list_mark)
check_markfil

,Year.Order Date,Category,Sub-Category,MAX.Quantity,SUM.Sales
0,2017,Furniture,Furnishings,7,48.8600
1,2017,Furniture,Tables,9,1706.1840
2,2017,Office Supplies,Appliances,5,114.9000
3,2017,Office Supplies,Art,4,15.8400
4,2017,Office Supplies,Binders,3,18.5040
5,2017,Office Supplies,Storage,6,721.3800
6,2017,Technology,Phones,6,2032.0560
7,2018,Furniture,Tables,5,957.5775
8,2018,Office Supplies,Appliances,5,68.8100
9,2018,Office Supplies,Binders,3,2.5440


In [218]:
def filter_add(frame_merge,row_list,column_list,select_topic):
    all_list = row_list+column_list
    with open('filterdata.json', 'r') as file_readjson:
        read_json = file_readjson.read()
        read_data = json.loads(read_json)
    name_col = list(frame_merge.keys())
    for path in select_topic:
        md5_hash = hashlib.md5()
        file_md5 = open(path, "rb")
        file_md5_read = file_md5.read()
        md5_hash.update(file_md5_read)
        code_md5 = md5_hash.hexdigest()
        if "filter" in list(read_data[code_md5].keys()):
            filter_key = list(read_data[code_md5]["filter"].keys())
            for fil_name in all_list:
                drill_split = fil_name.split("+")
                fil_split = drill_split[0].split(".")
                if fil_name in filter_key:
                    filter_list = read_data[code_md5]["filter"][fil_name]
                    if len(fil_split) > 1:
                        frame_merge = frame_merge.loc[frame_merge[drill_split[0]].isin(filter_list)]
                    else:
                        frame_merge = frame_merge.loc[frame_merge[drill_split[0]].isin(filter_list)]          
            new_frame = frame_merge                 
        else:
            new_frame = frame_merge
    return new_frame
data_addfil = filter_add(check_markfil,row,column,path)# "filter": {"Category": ["Furniture", "Technology"]
list(data_addfil['Category'].drop_duplicates())

['Furniture', 'Technology']

In [219]:
def filtermeasure_add(frame_merge,row_list,column_list,select_topic):

    all_list = row_list+column_list
    with open('filtermeasure.json', 'r') as file_readjson:
        read_json = file_readjson.read()
        read_data = json.loads(read_json)

    name_col = list(frame_merge.keys())
    for path in select_topic:
        md5_hash = hashlib.md5()
        file_md5 = open(path, "rb")
        file_md5_read = file_md5.read()
        md5_hash.update(file_md5_read)
        code_md5 = md5_hash.hexdigest()
        if "filter" in list(read_data[code_md5].keys()):
            for fil_name in name_col:
                drill_split = fil_name.split("+")
                fil_split = drill_split[0].split(".")
                filter_key = list(read_data[code_md5]["filter"].keys())
                if fil_name in filter_key:
                    filter_list = read_data[code_md5]["filter"][fil_name]
                    frame_merge = frame_merge.loc[(frame_merge[fil_name] >= float(filter_list[0])) & (frame_merge[fil_name] <= float(filter_list[1]))]
            new_frame = frame_merge                 
        else:
            new_frame = frame_merge
    return new_frame
data_addfilmeas = filtermeasure_add(data_addfil,row,column,path) # "filter": {"MAX.Quantity": ["2", "5"]   
data_addfilmeas

,Year.Order Date,Category,Sub-Category,MAX.Quantity,SUM.Sales
7,2018,Furniture,Tables,5,957.5775
11,2019,Furniture,Bookcases,2,261.9600
12,2019,Furniture,Chairs,3,731.9400


In [225]:
import unittest

class TesttableManage(unittest.TestCase):
    
    def test_read_data(self):
        path = ["C:/Users/ASUS/Documents/vs/16th/test20line.xls","C:/Users/ASUS/Documents/vs/16th/test_union.xlsx"]
        data_check = read_data(path)
        self.assertTrue(data_check['Category'][25],'Furniture')

    def test_dim_meas_rc(self):
        dimensions = ["Row ID","Order ID","Order Date","Ship Date","Ship Mode","Customer ID","Customer Name",
        "Segment","Country/Region","City","State","Postal Code","Region","Product ID","Category","Sub-Category","Product Name"]
        measure = ["Sales","Quantity","Discount","Profit"]
        row = ["Quantity","Category","Sub-Category"]
        column = ["Country/Region","City","Sales"]
        list_dim,list_meas,list_mark,list_filter = dim_meas_rc(row,column,dimensions,measure)
        self.assertTrue(list_meas[0],'Quantity') #["Quantity","Sales"]
    
    def test_check_date(self):
        path = ["C:/Users/ASUS/Documents/vs/16th/test20line.xls"]
        data_check = read_data(path)
        list_dim = ['Order Date','Category','Sub-Category']
        list_filter = ['Year','NONE','NONE']
        frame,list_dimfil  = check_date_filter(data_check,list_dim,list_filter)
        self.assertEqual(frame['Year.Order Date'][0],2019)

    def test_check_date2(self):
        path = ["C:/Users/ASUS/Documents/vs/16th/test20line.xls"]
        data_check = read_data(path)
        list_dim = ['Order Date','Category','Sub-Category']
        list_filter = ['Year','NONE','NONE']
        frame,list_dimfil  = check_date_filter(data_check,list_dim,list_filter)
        self.assertEqual(frame['Year.Order Date'][0],2010)
    
    def test_check_mark(self):
        path = ["C:/Users/ASUS/Documents/vs/16th/test20line.xls"]
        data_check = read_data(path)
        list_dim = ['Order Date','Category','Sub-Category']
        list_filter = ['Year','NONE','NONE']
        list_meas = ['Quantity','Sales']
        list_mark = ['MAX','SUM']
        check_markdate,list_dimfil  = check_date_filter(data_check,list_dim,list_filter)
        check_markfil = check_mark(check_markdate,list_dimfil,list_meas,list_mark)
        self.assertEqual(check_markfil['MAX.Quantity'][0],7) #7
    
    def test_filter_add(self):
        path = ["C:/Users/ASUS/Documents/vs/16th/test20line.xls"]
        data_check = read_data(path)
        row = ["Year.Order Date","MAX.Quantity","Category"]
        column = ["Sales","Sub-Category"]
        list_dim = ['Order Date','Category','Sub-Category']
        list_filter = ['Year','NONE','NONE']
        list_meas = ['Quantity','Sales']
        list_mark = ['MAX','SUM']
        check_markdate,list_dimfil  = check_date_filter(data_check,list_dim,list_filter)
        check_markfil = check_mark(check_markdate,list_dimfil,list_meas,list_mark)
        check_filadd = filter_add(check_markfil,row,column,path)
        self.assertTrue(len(list(check_filadd['Category'].drop_duplicates())),2) #2
      
    
        

In [226]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_check_date (__main__.TesttableManage) ... ok
test_check_date2 (__main__.TesttableManage) ... FAIL
test_check_mark (__main__.TesttableManage) ... ok
test_dim_meas_rc (__main__.TesttableManage) ... ok
test_filter_add (__main__.TesttableManage) ... C:\Users\ASUS\AppData\Local\Temp\ipykernel_17416\2982115053.py:55: ResourceWarning: unclosed file <_io.BufferedReader name='C:/Users/ASUS/Documents/vs/16th/test20line.xls'>
  check_filadd = filter_add(check_markdate,row,column,path)
ok
test_read_data (__main__.TesttableManage) ... ok

FAIL: test_check_date2 (__main__.TesttableManage)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_17416\2982115053.py", line 33, in test_check_date2
    self.assertEqual(frame['Year.Order Date'][0],2010)
AssertionError: 2019 != 2010

----------------------------------------------------------------------
Ran 6 tests in 0.219s

FAILED (failures=1)
